<a href="https://colab.research.google.com/github/ShoaibSheriff/Computer-Vision/blob/master/Direction_AVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch
!pip install torchvision

    100% |████████████████████████████████| 591.8MB 28kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x61f72000 @  0x7f7f6648f2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 2.3MB/s 
    100% |████████████████████████████████| 2.0MB 14.1MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_asd = pd.read_csv(r"drive/My Drive/ava/ava-human-searches_anno.csv")

In [6]:
df_asd.head(5)

,search-id,step-id,time,video-id,target-action
0,0,0,849.623600,Ag-pXiLrd48,clink glass
1,0,1,852.323675,Ag-pXiLrd48,clink glass
2,0,2,855.023750,Ag-pXiLrd48,clink glass
3,0,3,850.523625,Ag-pXiLrd48,clink glass
4,0,4,853.223700,Ag-pXiLrd48,clink glass


In [0]:
actions_to_be_considered = df_asd['target-action'].unique()

In [8]:
len(df_asd['target-action'].unique())

20

In [0]:
act_list = ["bend/bow (at the waist)", "crawl", "crouch/kneel", "dance", "fall down", "get up", "jump/leap", "lie/sleep", "martial art", "run/jog", "sit", "stand", "swim", "walk", "answer phone", "brush teeth", "carry/hold (an object)", "catch (an object)", "chop", "climb (e.g., a mountain)", "clink glass", "close (e.g., a door, a box)", "cook", "cut", "dig", "dress/put on clothing", "drink", "drive (e.g., a car, a truck)", "eat", "enter", "exit", "extract", "fishing", "hit (an object)", "kick (an object)", "lift/pick up", "listen (e.g., to music)", "open (e.g., a window, a car door)", "paint", "play board game", "play musical instrument", "play with pets", "point to (an object)", "press", "pull (an object)", "push (an object)", "put down", "read", "ride (e.g., a bike, a car, a horse)", "row boat", "sail boat", "shoot", "shovel", "smoke", "stir", "take a photo", "text on/look at a cellphone", "throw", "touch (an object)", "turn (e.g., a screwdriver)", "watch (e.g., TV)", "work on a computer", "write", "fight/hit (a person)", "give/serve (an object) to (a person)", "grab (a person)", "hand clap", "hand shake", "hand wave", "hug (a person)", "kick (a person)", "kiss (a person)", "lift (a person)", "listen to (a person)", "play with kids", "push (another person)", "sing to (e.g., self, a person, a group)", "take (an object) from (a person)", "talk to (e.g., self, a person, a group)", "watch (a person)"]

In [10]:
len(act_list)

80

In [0]:
act_dict = {}
for i in range(len(act_list)) :
  if act_list[i] in actions_to_be_considered :
    act_dict[i+1] = act_list[i]


In [0]:
videos_considered = df_asd['video-id'].unique()

In [13]:
len(df_asd['video-id'].unique())

138

In [0]:
MIN = 904

In [0]:
MAX =  1798

# Direction

In [16]:
import torchvision.models as models
resnet34 = models.resnet34(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:01<00:00, 46874544.67it/s]


In [0]:
import torch
import torch.nn as nn

resnet_binary = nn.Sequential(resnet34, nn.Sequential(nn.Linear(1000,2), nn.Softmax()))

In [18]:
resnet_binary

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [0]:
df_dummy = pd.read_csv(r"drive/My Drive/ava/ava_working_train_ids.csv", names  = ['ids'], index_col = False)

In [20]:
df_dummy.shape

(115, 1)

In [0]:
working_video_ids = df_dummy['ids'].unique()

In [0]:
df_ava_train = pd.read_csv(r"drive/My Drive/ava/ava_train_v1.0_v.csv", names  = ['video_id', 'middle_frame_timestamp', 'x1', 'y1', 'x2', 'y2', 'action_id'], index_col = False)

In [0]:
df_ava_train_working_only = df_ava_train[(df_ava_train['video_id'].isin(working_video_ids))]

In [24]:
df_ava_train_working_only.head(5)

,video_id,middle_frame_timestamp,x1,y1,x2,y2,action_id
0,-5KQ66BBWC4,904,0.217,0.008,0.982,0.966,12
1,-5KQ66BBWC4,904,0.217,0.008,0.982,0.966,80
2,-5KQ66BBWC4,907,0.062,0.074,0.234,0.926,12
3,-5KQ66BBWC4,907,0.190,0.109,0.396,0.995,12
4,-5KQ66BBWC4,907,0.420,0.115,0.616,0.883,14


In [25]:
len(df_ava_train_working_only)

123877

In [26]:
act_list = [*act_dict]
act_list

[4, 7, 9, 13, 21, 27, 28, 29, 41, 52, 54, 56, 62, 63, 67, 68, 70, 72, 76, 77]

In [0]:
# df_ava_train_working_only[(df_ava_train_working_only['action_id'] == 4) & (df_ava_train_working_only['video_id']  == 'tghXjom3120')].sort_values('middle_frame_timestamp').head(100)
# {904: 222, 905: 221, 906: 220, 907: 219, 908: 218, 909: 217, 910: 216, 911: 215, 912: 214, 913: 213, 914: 212, 915: 211, 916: 210, 917: 209, 918: 208, 919: 207, 920: 206, 921: 205, 922: 204, 923: 203, 924: 202, 925: 201, 926: 200, 927: 199, 928: 198, 929: 197, 930: 196, 931: 195, 932: 194, 933: 193, 934: 192, 935: 191, 936: 190, 937: 189, 938: 188, 939: 187, 940: 186, 941: 185, 942: 184, 943: 183, 944: 182, 945: 181, 946: 180, 947: 179, 948: 178, 949: 177, 950: 176, 951: 175, 952: 174, 953: 173, 954: 172, 955: 171, 956: 170, 957: 169, 958: 168, 959: 167, 960: 166, 961: 165, 962: 164, 963: 163, 964: 162, 965: 161, 966: 160, 967: 159, 968: 158, 969: 157, 970: 156, 971: 155, 972: 154, 973: 153, 974: 152, 975: 151, 976: 150, 977: 149, 978: 148, 979: 147, 980: 146, 981: 145, 982: 144, 983: 143, 984: 142, 985: 141, 986: 140, 987: 139, 988: 138, 989: 137, 990: 136, 991: 135, 992: 134, 993: 133, 994: 132, 995: 131, 996: 130, 997: 129, 998: 128, 999: 127, 1000: 126, 1001: 125, 1002: 124, 1003: 123, 1004: 122, 1005: 121, 1006: 120, 1007: 119, 1008: 118, 1009: 117, 1010: 116, 1011: 115, 1012: 114, 1013: 113, 1014: 112, 1015: 111, 1016: 110, 1017: 109, 1018: 108, 1019: 107, 1020: 106, 1021: 105, 1022: 104, 1023: 103, 1024: 102, 1025: 101, 1026: 100, 1027: 99, 1028: 98, 1029: 97, 1030: 96, 1031: 95, 1032: 94, 1033: 93, 1034: 92, 1035: 91, 1036: 90, 1037: 89, 1038: 88, 1039: 87, 1040: 86, 1041: 85, 1042: 84, 1043: 83, 1044: 82, 1045: 81, 1046: 80, 1047: 79, 1048: 78, 1049: 77, 1050: 76, 1051: 75, 1052: 74, 1053: 73, 1054: 72, 1055: 71, 1056: 70, 1057: 69, 1058: 68, 1059: 67, 1060: 66, 1061: 65, 1062: 64, 1063: 63, 1064: 62, 1065: 61, 1066: 60, 1067: 59, 1068: 58, 1069: 57, 1070: 56, 1071: 55, 1072: 54, 1073: 53, 1074: 52, 1075: 51, 1076: 50, 1077: 49, 1078: 48, 1079: 47, 1080: 46, 1081: 45, 1082: 44, 1083: 43, 1084: 42, 1085: 41, 1086: 40, 1087: 39, 1088: 38, 1089: 37, 1090: 36, 1091: 35, 1092: 34, 1093: 33, 1094: 32, 1095: 31, 1096: 30, 1097: 29, 1098: 28, 1099: 27, 1100: 26, 1101: 25, 1102: 24, 1103: 23, 1104: 22, 1105: 21, 1106: 20, 1107: 19, 1108: 18, 1109: 17, 1110: 16, 1111: 15, 1112: 14, 1113: 13, 1114: 12, 1115: 11, 1116: 10, 1117: 9, 1118: 8, 1119: 7, 1120: 6, 1121: 5, 1122: 4, 1123: 3, 1124: 2, 1125: 1, 1193: -1, 1194: -2, 1195: -3, 1196: 2, 1197: 1, 1202: -1, 1203: -2, 1204: -3, 1205: -4, 1206: -5, 1207: -6, 1208: -7, 1209: -8, 1210: -9, 1211: -10, 1212: -11, 1213: -12, 1214: -13, 1215: -14, 1216: -15, 1217: -16, 1218: -17, 1219: -18, 1220: -19, 1221: -20, 1222: -21, 1223: -22, 1224: -23, 1225: -24, 1226: -25, 1227: -26, 1228: -27, 1229: -28, 1230: -29, 1231: -30, 1232: -31, 1233: -32, 1234: -33, 1235: -34, 1236: -35, 1237: -36, 1238: -37, 1239: -38, 1240: -39, 1241: -40, 1242: -41, 1243: -42, 1244: -43, 1245: -44, 1246: -45, 1247: -46, 1248: -47, 1249: -48, 1250: -49, 1251: -50, 1252: -51, 1253: -52, 1254: -53, 1255: -54, 1256: -55, 1257: -56, 1258: -57, 1259: -58, 1260: -59, 1261: -60, 1262: -61, 1263: -62, 1264: -63, 1265: -64, 1266: -65, 1267: -66, 1268: -67, 1269: -68, 1270: -69, 1271: -70, 1272: -71, 1273: -72, 1274: -73, 1275: -74, 1276: -75, 1277: -76, 1278: -77, 1279: -78, 1280: -79, 1281: -80, 1282: -81, 1283: -82, 1284: -83, 1285: -84, 1286: -85, 1287: -86, 1288: -87, 1289: -88, 1290: -89, 1291: -90, 1292: -91, 1293: -92, 1294: -93, 1295: -94, 1296: -95, 1297: -96, 1298: -97, 1299: -98, 1300: -99, 1301: -100, 1302: -101, 1303: -102, 1304: -103, 1305: -104, 1306: -105, 1307: -106, 1308: -107, 1309: -108, 1310: -109, 1311: -110, 1312: -111, 1313: -112, 1314: -113, 1315: -114, 1316: -115, 1317: -116, 1318: -117, 1319: -118, 1320: -119, 1321: -120, 1322: -121, 1323: -122, 1324: -123, 1325: -124, 1326: -125, 1327: -126, 1328: -127, 1329: -128, 1330: -129, 1331: -130, 1332: -131, 1333: -132, 1334: -133, 1335: -134, 1336: -135, 1337: -136, 1338: -137, 1339: -138, 1340: -139, 1341: -140, 1342: -141, 1343: -142, 1344: -143, 1345: -144, 1346: -145, 1347: -146, 1348: -147, 1349: -148, 1350: -149, 1351: -150, 1352: -151, 1353: -152, 1354: -153, 1355: -154, 1356: -155, 1357: -156, 1358: -157, 1359: -158, 1360: -159, 1361: -160, 1362: -161, 1363: -162, 1364: -163, 1365: -164, 1366: -165, 1367: -166, 1368: -167, 1369: -168, 1370: -169, 1371: -170, 1372: -171, 1373: -172, 1374: -173, 1375: -174, 1376: -175, 1377: -176, 1378: -177, 1379: -178, 1380: -179, 1381: -180, 1382: -181, 1383: -182, 1384: -183, 1385: -184, 1386: -185, 1387: -186, 1388: -187, 1389: -188, 1390: -189, 1391: -190, 1392: -191, 1393: -192, 1394: -193, 1395: -194, 1396: -195, 1397: -196, 1398: -197, 1399: -198, 1400: -199, 1401: -200, 1402: -201, 1403: -202, 1404: -203, 1405: -204, 1406: -205, 1407: -206, 1408: -207, 1409: -208, 1410: -209, 1411: -210, 1412: -211, 1413: -212, 1414: -213, 1415: -214, 1416: -215, 1417: -216, 1418: -217, 1419: -218, 1420: -219, 1421: -220, 1422: -221, 1423: -222, 1424: -223, 1425: -224, 1426: -225, 1427: -226, 1428: -227, 1429: -228, 1430: -229, 1431: -230, 1432: -231, 1433: -232, 1434: -233, 1435: -234, 1436: -235, 1437: -236, 1438: -237, 1439: -238, 1440: -239, 1441: -240, 1442: -241, 1443: -242, 1444: -243, 1445: -244, 1446: -245, 1447: -246, 1448: -247, 1449: -248, 1450: -249, 1451: -250, 1452: -251, 1453: -252, 1454: -253, 1455: -254, 1456: -255, 1457: -256, 1458: -257, 1459: -258, 1460: -259, 1461: -260, 1462: -261, 1463: -262, 1464: -263, 1465: -264, 1466: -265, 1467: -266, 1468: -267, 1469: -268, 1470: -269, 1471: -270, 1472: -271, 1473: -272, 1474: -273, 1475: -274, 1476: -275, 1477: -276, 1478: -277, 1479: -278, 1480: -279, 1481: -280, 1482: -281, 1483: -282, 1484: -283, 1485: -284, 1486: -285, 1487: -286, 1488: -287, 1489: -288, 1490: -289, 1491: -290, 1492: -291, 1493: -292, 1494: -293, 1495: -294, 1496: -295, 1497: -296, 1498: -297, 1499: -298, 1500: 298, 1501: 297, 1502: 296, 1503: 295, 1504: 294, 1505: 293, 1506: 292, 1507: 291, 1508: 290, 1509: 289, 1510: 288, 1511: 287, 1512: 286, 1513: 285, 1514: 284, 1515: 283, 1516: 282, 1517: 281, 1518: 280, 1519: 279, 1520: 278, 1521: 277, 1522: 276, 1523: 275, 1524: 274, 1525: 273, 1526: 272, 1527: 271, 1528: 270, 1529: 269, 1530: 268, 1531: 267, 1532: 266, 1533: 265, 1534: 264, 1535: 263, 1536: 262, 1537: 261, 1538: 260, 1539: 259, 1540: 258, 1541: 257, 1542: 256, 1543: 255, 1544: 254, 1545: 253, 1546: 252, 1547: 251, 1548: 250, 1549: 249, 1550: 248, 1551: 247, 1552: 246, 1553: 245, 1554: 244, 1555: 243, 1556: 242, 1557: 241, 1558: 240, 1559: 239, 1560: 238, 1561: 237, 1562: 236, 1563: 235, 1564: 234, 1565: 233, 1566: 232, 1567: 231, 1568: 230, 1569: 229, 1570: 228, 1571: 227, 1572: 226, 1573: 225, 1574: 224, 1575: 223, 1576: 222, 1577: 221, 1578: 220, 1579: 219, 1580: 218, 1581: 217, 1582: 216, 1583: 215, 1584: 214, 1585: 213, 1586: 212, 1587: 211, 1588: 210, 1589: 209, 1590: 208, 1591: 207, 1592: 206, 1593: 205, 1594: 204, 1595: 203, 1596: 202, 1597: 201, 1598: 200, 1599: 199, 1600: 198, 1601: 197, 1602: 196, 1603: 195, 1604: 194, 1605: 193, 1606: 192, 1607: 191, 1608: 190, 1609: 189, 1610: 188, 1611: 187, 1612: 186, 1613: 185, 1614: 184, 1615: 183, 1616: 182, 1617: 181, 1618: 180, 1619: 179, 1620: 178, 1621: 177, 1622: 176, 1623: 175, 1624: 174, 1625: 173, 1626: 172, 1627: 171, 1628: 170, 1629: 169, 1630: 168, 1631: 167, 1632: 166, 1633: 165, 1634: 164, 1635: 163, 1636: 162, 1637: 161, 1638: 160, 1639: 159, 1640: 158, 1641: 157, 1642: 156, 1643: 155, 1644: 154, 1645: 153, 1646: 152, 1647: 151, 1648: 150, 1649: 149, 1650: 148, 1651: 147, 1652: 146, 1653: 145, 1654: 144, 1655: 143, 1656: 142, 1657: 141, 1658: 140, 1659: 139, 1660: 138, 1661: 137, 1662: 136, 1663: 135, 1664: 134, 1665: 133, 1666: 132, 1667: 131, 1668: 130, 1669: 129, 1670: 128, 1671: 127, 1672: 126, 1673: 125, 1674: 124, 1675: 123, 1676: 122, 1677: 121, 1678: 120, 1679: 119, 1680: 118, 1681: 117, 1682: 116, 1683: 115, 1684: 114, 1685: 113, 1686: 112, 1687: 111, 1688: 110, 1689: 109, 1690: 108, 1691: 107, 1692: 106, 1693: 105, 1694: 104, 1695: 103, 1696: 102, 1697: 101, 1698: 100, 1699: 99, 1700: 98, 1701: 97, 1702: 96, 1703: 95, 1704: 94, 1705: 93, 1706: 92, 1707: 91, 1708: 90, 1709: 89, 1710: 88, 1711: 87, 1712: 86, 1713: 85, 1714: 84, 1715: 83, 1716: 82, 1717: 81, 1718: 80, 1719: 79, 1720: 78, 1721: 77, 1722: 76, 1723: 75, 1724: 74, 1725: 73, 1726: 72, 1727: 71, 1728: 70, 1729: 69, 1730: 68, 1731: 67, 1732: 66, 1733: 65, 1734: 64, 1735: 63, 1736: 62, 1737: 61, 1738: 60, 1739: 59, 1740: 58, 1741: 57, 1742: 56, 1743: 55, 1744: 54, 1745: 53, 1746: 52, 1747: 51, 1748: 50, 1749: 49, 1750: 48, 1751: 47, 1752: 46, 1753: 45, 1754: 44, 1755: 43, 1756: 42, 1757: 41, 1758: 40, 1759: 39, 1760: 38, 1761: 37, 1762: 36, 1763: 35, 1764: 34, 1765: 33, 1766: 32, 1767: 31, 1768: 30, 1769: 29, 1770: 28, 1771: 27, 1772: 26, 1773: 25, 1774: 24, 1775: 23, 1776: 22, 1777: 21, 1778: 20, 1779: 19, 1780: 18, 1781: 17, 1782: 16, 1783: 15, 1784: 14, 1785: 13, 1786: 12, 1787: 11, 1788: 10, 1789: 9, 1790: 8, 1791: 7, 1792: 6, 1793: 5, 1794: 4, 1795: 3, 1796: 2, 1797: 1}


In [27]:
# dict that contains left and right timestamp for all actions
actions_videos_left_right_dicts = {}

for act in act_list :
# for i in range(1) :

  print(act)
  df_train_action_cur = df_ava_train_working_only[(df_ava_train_working_only['action_id'] == act)]
  
  # dict that contains left and right timestamp for this action
  df_train_action_l = {}
  df_train_action_r = {}
  
  for video in df_train_action_cur['video_id'].unique() :
    print (video)
    
    df_train_action_l[video] = []
    df_train_action_r[video] = [] 
    
    df_train_action_video_cur = df_train_action_cur[(df_train_action_cur['video_id'] == video)]
    list_of_timestamp_with_action = df_train_action_video_cur['middle_frame_timestamp'].tolist()
  #   df_train_action_i[list_of_timestamp_with_actions] = 1
#     print(list_of_timestamp_with_actions)
  
    df_train_action_i = {}

    left_closest = 1111111111

    for time_stamp in list(range(MIN, MAX + 1)) :
      if time_stamp in list_of_timestamp_with_action :
        left_closest = time_stamp
        continue
      if (time_stamp - 1 in list_of_timestamp_with_action) & (time_stamp + 2 in list_of_timestamp_with_action):
        left_closest = time_stamp
        continue
      if (time_stamp - 2 in list_of_timestamp_with_action) & (time_stamp + 1 in list_of_timestamp_with_action):
        left_closest = time_stamp
        continue

      df_train_action_i[time_stamp] = left_closest - time_stamp


    right_closest = 1111111111  

    for time_stamp in list(range(MAX, MIN - 1, -1)) :
      if time_stamp in list_of_timestamp_with_action :
        right_closest = time_stamp
        continue
      if (time_stamp - 1 in list_of_timestamp_with_action) & (time_stamp + 2 in list_of_timestamp_with_action):
        right_closest = time_stamp
        continue
      if (time_stamp - 2 in list_of_timestamp_with_action) & (time_stamp + 1 in list_of_timestamp_with_action):
        right_closest = time_stamp
        continue

      if (abs(df_train_action_i[time_stamp]) > (right_closest - time_stamp)) :
        df_train_action_i[time_stamp] = right_closest - time_stamp

      
      
#     print(df_train_action_i)
    for timestamps_for_train in list([*df_train_action_i]) :
      if (df_train_action_i[timestamps_for_train] > 0) :
        df_train_action_r[video].append(timestamps_for_train)
      else :
        df_train_action_l[video].append(timestamps_for_train)
        
  actions_videos_left_right_dicts[act] = [df_train_action_l, df_train_action_r]
# print(len(action_video_left_right_dict[4]))
        

4
-FaXLcSFjUI
5BDj0ow5hnA
5YPjcdLbs5g
8aMv-ZGD4ic
914yZXz-iRs
9F2voT6QWvQ
AYebXQ8eUkM
B1MAUxpKaV8
CZ2NP8UsPuE
Db19rWN5BGo
F3dPH6Xqf5M
HymKCzQJbB8
J1jDc2rTJlg
KWoSGtglCms
Kb1fduj-jdY
Ma2hgTmveKQ
NEQ7Wpf-EtI
O_NYCUhZ9zw
PNZQ2UJfyQE
S0tkhGJjwLA
T-Fc9ctuNVI
VsYPP2I0aUQ
XV_FF3WC7kA
_mAfwH6i90E
b5pRYl_djbs
cKA-qeZuH_w
cWYJHb25EVs
fpprSy6AzKk
l-jxh8gpxuY
o4xQ-BEa3Ss
oq_bufAhyl8
plkJ45_-pMk
qx2vAO5ofmo
sUVhd0YTKgw
tghXjom3120
u1ltv6r14KQ
xO4ABy2iOQA
xmqSaQPzL1E
7
-5KQ66BBWC4
-FaXLcSFjUI
4gVsDd8PV9U
55Ihr6uVIDA
914yZXz-iRs
9F2voT6QWvQ
B1MAUxpKaV8
F3dPH6Xqf5M
J4bt4y9ShTA
KVq6If6ozMY
KWoSGtglCms
Kb1fduj-jdY
Ma2hgTmveKQ
N5UD8FGzDek
QCLQYnt3aMo
U6m3kNFjdTs
WVde9pyaHg4
XV_FF3WC7kA
_a9SWtcaNj8
_mAfwH6i90E
cKA-qeZuH_w
fD6VkIRlIRI
fNcxxBjEOgw
iSlDMboCSao
lDmLcWWBp1E
o4xQ-BEa3Ss
oD_wxyTHJ2I
rFgb2ECMcrY
rUYsoIIE37A
rk8Xm0EAOWs
tghXjom3120
x-6CtPWVi6E
xmqSaQPzL1E
y7ncweROe9U
yMtGmGa8KZ0
9
-5KQ66BBWC4
4gVsDd8PV9U
8aMv-ZGD4ic
914yZXz-iRs
9F2voT6QWvQ
B1MAUxpKaV8
HymKCzQJbB8
KWoSGtglCms
QCLQYnt3aMo
U6m3kNFjdT

In [28]:
print([*actions_videos_left_right_dicts])


[4, 7, 9, 13, 21, 27, 28, 29, 41, 52, 54, 56, 62, 63, 67, 68, 70, 72, 76, 77]


In [50]:
action_training_dict = {}
for action in list([*actions_videos_left_right_dicts]) :
  videos = []
  timestamps = []
  labels = []
  
  all_lefts, all_rights = actions_videos_left_right_dicts[action]
  
  for video_l in list([*all_lefts]) :
    print("drive/My Drive/ava/video_resized/" + str(video_l) + ".mp4")
    video = "drive/My Drive/ava/video_resized/" + str(video_l) + ".mp4"

    print(len(all_lefts[video_l]))
    for left_timestamp in all_lefts[video_l] :
      videos.append(video)
      timestamps.append((left_timestamp - 904) * 1000)
      labels.append(0)


  for video_r in list([*all_rights]) :
    print("drive/My Drive/ava/video_resized/" + str(video_r) + ".mp4")
    video = "drive/My Drive/ava/video_resized/" + str(video_r) + ".mp4"

    print(len(all_rights[video_r]))
    for right_timestamp in all_rights[video_l] :
      
      videos.append(video)
      timestamps.append((right_timestamp - 904) * 1000)
      labels.append(1)
  action_training_dict[action] = [videos, timestamps, labels]

drive/My Drive/ava/video_resized/-FaXLcSFjUI.mp4
775
drive/My Drive/ava/video_resized/5BDj0ow5hnA.mp4
518
drive/My Drive/ava/video_resized/5YPjcdLbs5g.mp4
324
drive/My Drive/ava/video_resized/8aMv-ZGD4ic.mp4
490
drive/My Drive/ava/video_resized/914yZXz-iRs.mp4
30
drive/My Drive/ava/video_resized/9F2voT6QWvQ.mp4
726
drive/My Drive/ava/video_resized/AYebXQ8eUkM.mp4
630
drive/My Drive/ava/video_resized/B1MAUxpKaV8.mp4
36
drive/My Drive/ava/video_resized/CZ2NP8UsPuE.mp4
788
drive/My Drive/ava/video_resized/Db19rWN5BGo.mp4
346
drive/My Drive/ava/video_resized/F3dPH6Xqf5M.mp4
683
drive/My Drive/ava/video_resized/HymKCzQJbB8.mp4
374
drive/My Drive/ava/video_resized/J1jDc2rTJlg.mp4
257
drive/My Drive/ava/video_resized/KWoSGtglCms.mp4
336
drive/My Drive/ava/video_resized/Kb1fduj-jdY.mp4
442
drive/My Drive/ava/video_resized/Ma2hgTmveKQ.mp4
373
drive/My Drive/ava/video_resized/NEQ7Wpf-EtI.mp4
27
drive/My Drive/ava/video_resized/O_NYCUhZ9zw.mp4
813
drive/My Drive/ava/video_resized/PNZQ2UJfyQE.mp4


In [0]:
from torch.utils.data import DataLoader,sampler,Dataset

class ActionSearchDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, action, act_training_data):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.action = action
        self.act_training_data = act_training_data

    def __len__(self):
        videos, timestamps, labels = self.act_training_data[action]
        return len(videos)

    def __getitem__(self, idx):
      
        videos, timestamps, labels = self.act_training_data[action]
        cap = cv2.VideoCapture(videos[idx])
        
        cap.set(cv2.CAP_PROP_POS_MSEC,timestamps[idx]) 
        ret, frame = cap.read()
        
        train_transforms = transforms.Compose([
          transforms.ToTensor(), 
          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        
        frame = train_transforms(frame)
        label = labels[idx]
        sample = {'image': frame, 'label': label}

        return sample

In [52]:
as_dataset=ActionSearchDataset(action = 4, act_training_data = action_training_dict)

for i in range(10):
    sample=as_dataset[i]
    print(sample['image'].shape)
    print(sample['label'])

torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0


In [53]:
as_dataset_batches = DataLoader(as_dataset, batch_size=32,
                        shuffle=True, num_workers=4)
for i,sample in enumerate(as_dataset_batches):
   
    print(i,sample['image'].shape, sample['label'])
    if i>10: 
        break

0 torch.Size([32, 3, 224, 224]) tensor([1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0])
1 torch.Size([32, 3, 224, 224]) tensor([1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
        1, 0, 1, 1, 0, 0, 1, 1])
2 torch.Size([32, 3, 224, 224]) tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
        1, 0, 0, 1, 1, 0, 1, 0])
3 torch.Size([32, 3, 224, 224]) tensor([1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 0, 0, 1, 1])
4 torch.Size([32, 3, 224, 224]) tensor([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 0])
5 torch.Size([32, 3, 224, 224]) tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 1, 0, 0])
6 torch.Size([32, 3, 224, 224]) tensor([0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
        0, 1, 1, 1

In [0]:
def train(model, loss_fn, optimizer, dataloader, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, sample in enumerate(dataloader):
            x_var = Variable(sample['image'].float().cuda())
            y_var = Variable(sample['label'].long().cuda())

            scores = model(x_var)
            
            loss = loss_fn(scores, y_var)
            if (t + 1) % 100 == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [55]:
import copy
gpu_dtype = torch.cuda.FloatTensor

resnet_binary = copy.deepcopy(resnet_binary)
resnet_binary.cuda()

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [56]:
# This sets the model in "training" mode. 
# This is relevant for some layers that may have different behavior
# in training mode vs testing mode, such as Dropout and BatchNorm. 
loss_fn = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(resnet_binary.parameters(), lr = 1e-4)

torch.cuda.random.manual_seed(12345)
resnet_binary.train() 
train(resnet_binary, loss_fn, optimizer, as_dataset_batches, num_epochs=15) 

Starting epoch 1 / 10


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


t = 10, loss = 0.6386
t = 20, loss = 0.7484
t = 30, loss = 0.6566
t = 40, loss = 0.6163
t = 50, loss = 0.6883
t = 60, loss = 0.7820
t = 70, loss = 0.6269
t = 80, loss = 0.7195
t = 90, loss = 0.8132
t = 100, loss = 0.7196
t = 110, loss = 0.7195
t = 120, loss = 0.6258
t = 130, loss = 0.5008
t = 140, loss = 0.6570
t = 150, loss = 0.6257
t = 160, loss = 0.5004
t = 170, loss = 0.6573
t = 180, loss = 0.5633
t = 190, loss = 0.5032
t = 200, loss = 0.8155
t = 210, loss = 0.5634
t = 220, loss = 0.6559
t = 230, loss = 0.7820
t = 240, loss = 0.5632
t = 250, loss = 0.5947
t = 260, loss = 0.5634
t = 270, loss = 0.5946
t = 280, loss = 0.7196
t = 290, loss = 0.5953
t = 300, loss = 0.8132
t = 310, loss = 0.6882
t = 320, loss = 0.5945
t = 330, loss = 0.6881
t = 340, loss = 0.7185
t = 350, loss = 0.6269
t = 360, loss = 0.7507
t = 370, loss = 0.5945
t = 380, loss = 0.5633
t = 390, loss = 0.6570
t = 400, loss = 0.5945
t = 410, loss = 0.5633
t = 420, loss = 0.6883
t = 430, loss = 0.5320
t = 440, loss = 0.84

In [0]:
def check_accuracy(model, loader):
    '''
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')  
    '''
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for t, sample in enumerate(loader):
        
        x_var = Variable(sample['image'].float().cuda())
        y_var = Variable(sample['label'].long().cuda())
        
        y_var=y_var.cpu()
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        #print(preds)
        #print(y_var)
        num_correct += (preds.numpy() == y_var.numpy()).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [58]:

check_accuracy(resnet_binary, as_dataset_batches)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Got 30404 / 44726 correct (67.98)


In [1]:
torch.save(resnet_binary.state_dict(), "act4.h5")

NameError: ignored

In [0]:

# import random

# width = 224
# height = 224
# Xl = np.zeros((0, height, width, 3))
# Xr = np.zeros((0, height, width, 3))

# # y = np.zeros((33592,2))

# #Train resnet
# # for action in [[*actions_videos_left_right_dicts][0]] :
# # action = [*actions_videos_left_right_dicts][0]
# action = 7
# all_lefts, all_rights = actions_videos_left_right_dicts[action]
# print(action)
# print(len([*all_lefts]))
# print(len([*all_rights]))

# for video_l in list([*all_lefts]) :
#   print("drive/My Drive/ava/video_resized/" + str(video_l) + ".mp4")
#   cap = cv2.VideoCapture("drive/My Drive/ava/video_resized/" + str(video_l) + ".mp4")

#   print(len(all_lefts[video_l]))
#   for left_timestamp in all_lefts[video_l] :
# #     print(left_timestamp)


#     zufall = random.randrange(0, 100)
#     if zufall > 39:
#       continue;
      
#     cap.set(cv2.CAP_PROP_POS_MSEC,(left_timestamp - 904) * 1000) 

#     ret, frame = cap.read()


# for video_r in list([*all_rights]) :
#   print("drive/My Drive/ava/video_resized/" + str(video_r) + ".mp4")
#   cap = cv2.VideoCapture("drive/My Drive/ava/video_resized/" + str(video_r) + ".mp4")

#   print(len(all_rights[video_r]))
#   for right_timestamp in all_rights[video_l] :

  
#     zufall = random.randrange(0, 100)
#     if zufall > 39:
#       continue;

#     cap.set(cv2.CAP_PROP_POS_MSEC,(right_timestamp - 904) * 1000) 
#     ret, frame = cap.read()
#     Xr = np.vstack((Xr, frame[np.newaxis]))


In [0]:
# import torch.optim as optim
# from torch.autograd import Variable

# resnet_binary.cuda()
# dtype = torch.cuda.FloatTensor
# loss_fn = nn.CrossEntropyLoss().type(dtype)
# optimizer = optim.SGD(resnet_binary.parameters(), lr=1e-3, momentum = 0.9)